In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Seeds

In [3]:
def fix_random_seeds(seed=12345):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)


fix_random_seeds()

# Configuration

In [4]:
model_config = {
    'init_lr': 0.01,
    'batch_size': 32,
    'epochs': 25,
    'empty_image_representation': 'zero_matrix',  #  zero_matrix, torch_empty
    'dino_architecture': 'small',
    'dataset': 'polyvore_63eb50dc58d97415384467bef7b3c9e1bd6c96e06ad19571b6bc15e9dd5af262.parquet',
    # 'output_logistics_thresholds': [0.8, 0.7, 0.6, 0.5, 0.4, 0.3],
    'model_forward_version': 'fast',  # slow, fast
    'hidden_layer_neuron_count': 64
}

dataset_folder_root_path = '../datasets'
dataset_path = f'{dataset_folder_root_path}/imageBasedModel/polyvore/{model_config["dataset"]}'

# Load the dataset and DINO model

In [5]:
Train_df = pd.read_parquet('../datasets/imageBasedModel/polyvore/polyvore_train_eb26e630100b98397deda54fa4a0bb95929479bc30e83cbfa72424b7c1e6e178.parquet')
Test_df = pd.read_parquet('../datasets/imageBasedModel/polyvore/polyvore_test_eb26e630100b98397deda54fa4a0bb95929479bc30e83cbfa72424b7c1e6e178.parquet')

In [6]:
df = pd.concat([Train_df, Test_df])

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6723 entries, 214761192 to 208373269
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Innerwear_imagePath           6723 non-null   object
 1   Bottomwear_imagePath          6723 non-null   object
 2   Accessoire_imagePath          5631 non-null   object
 3   Shoes_imagePath               3984 non-null   object
 4   Outerwear_imagePath           2179 non-null   object
 5   Innerwear_imagePath 256x256   6723 non-null   object
 6   Bottomwear_imagePath 256x256  6723 non-null   object
 7   Accessoire_imagePath 256x256  5631 non-null   object
 8   Shoes_imagePath 256x256       3984 non-null   object
 9   Outerwear_imagePath 256x256   2179 non-null   object
 10  valid_outfit                  6723 non-null   int64 
dtypes: int64(1), object(10)
memory usage: 630.3+ KB


In [8]:
# DINOv2
dinov2 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
dinov2 = dinov2.eval().to(device)
#dinov2_vitb14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14')
#dinov2_vitl14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitl14')
#dinov2_vitg14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitg14')

Using cache found in /Users/luca/.cache/torch/hub/facebookresearch_dinov2_main
/Users/luca/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/Users/luca/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/Users/luca/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [9]:
dinov2 = dinov2.eval().to(device)

# Load a test image

In [10]:
image_path = '../datasets/images_256x256/100119331/1.jpg'
input_image = Image.open(image_path)

transform = T.Compose([
    T.Resize(256, interpolation=T.InterpolationMode.BICUBIC),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

image = transform(input_image).to(device)

In [11]:
with torch.no_grad():
    features = dinov2(image.unsqueeze(0))

print(features)
print(features.shape)

tensor([[ 3.4772,  1.7773,  1.4635,  2.2478, -3.1532, -1.9697,  0.0358, -1.5149,
          3.5743, -2.1269,  1.0779, -0.3922, -1.1458, -1.3456,  5.0471,  3.7748,
          2.0318, -2.5745,  0.3614,  0.1399,  0.0674,  0.7749, -1.2847,  2.1297,
          1.2446, -2.6839,  1.5353,  2.9477,  0.8563,  2.3525, -2.8953,  0.0990,
         -0.4088, -2.4031, -0.5959,  3.5995,  2.6785, -2.3534, -1.0936, -2.2574,
          1.4675, -1.6633,  3.9569,  1.5211, -1.5145, -0.2431, -1.5367, -1.5053,
         -2.2562, -1.8115,  4.4599,  2.6838, -3.3050, -1.4802,  2.6929, -3.9444,
          2.6213, -1.5817, -4.5009, -1.3996,  0.9181,  2.2265, -1.0144, -2.1989,
         -0.7569,  5.8527,  3.9435, -4.7644,  0.4165,  1.5716,  0.8196,  1.1729,
          0.3606, -1.8431, -4.0229,  0.3466, -2.0093,  3.5627, -1.9622, -0.2825,
         -1.1377, -1.6338, -0.5705,  3.9196, -0.2342, -4.9130, -0.5169, -0.4783,
         -1.2212,  2.1530,  1.5316,  3.2916,  1.2449, -0.1067,  0.0386,  1.4902,
         -1.3760, -0.9201, -

In [12]:
def dinov2_classifier(img_url):
    
    if img_url is None:
        if model_config["empty_image_representation"] == "zero_matrix":
            return torch.zeros(3, 224, 224)
        elif model_config["empty_image_representation"] == "torch_empty":
            return torch.empty(3, 224, 224)
    
    image = Image.open(img_url)

    # Preprocess the image
    transform = T.Compose([
        T.Resize(256, interpolation=T.InterpolationMode.BICUBIC),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ])
    image = transform(image)

    # Move the image to the GPU if available
    image = image.to(device)

    # Extract the features
    with torch.no_grad():
        features = dinov2(image.unsqueeze(0)) #
        #features = torch.squeeze(feature_extraction_model(image.unsqueeze(0)))

    # Print the features
    return features

In [13]:
dinov2_classifier(image_path)

tensor([[ 3.4772,  1.7773,  1.4635,  2.2478, -3.1532, -1.9697,  0.0358, -1.5149,
          3.5743, -2.1269,  1.0779, -0.3922, -1.1458, -1.3456,  5.0471,  3.7748,
          2.0318, -2.5745,  0.3614,  0.1399,  0.0674,  0.7749, -1.2847,  2.1297,
          1.2446, -2.6839,  1.5353,  2.9477,  0.8563,  2.3525, -2.8953,  0.0990,
         -0.4088, -2.4031, -0.5959,  3.5995,  2.6785, -2.3534, -1.0936, -2.2574,
          1.4675, -1.6633,  3.9569,  1.5211, -1.5145, -0.2431, -1.5367, -1.5053,
         -2.2562, -1.8115,  4.4599,  2.6838, -3.3050, -1.4802,  2.6929, -3.9444,
          2.6213, -1.5817, -4.5009, -1.3996,  0.9181,  2.2265, -1.0144, -2.1989,
         -0.7569,  5.8527,  3.9435, -4.7644,  0.4165,  1.5716,  0.8196,  1.1729,
          0.3606, -1.8431, -4.0229,  0.3466, -2.0093,  3.5627, -1.9622, -0.2825,
         -1.1377, -1.6338, -0.5705,  3.9196, -0.2342, -4.9130, -0.5169, -0.4783,
         -1.2212,  2.1530,  1.5316,  3.2916,  1.2449, -0.1067,  0.0386,  1.4902,
         -1.3760, -0.9201, -

In [14]:
Columns_to_embed = ['Innerwear_imagePath 256x256', 'Bottomwear_imagePath 256x256', 'Accessoire_imagePath 256x256',
       'Shoes_imagePath 256x256', 'Outerwear_imagePath 256x256']

In [15]:
df.columns

Index(['Innerwear_imagePath', 'Bottomwear_imagePath', 'Accessoire_imagePath',
       'Shoes_imagePath', 'Outerwear_imagePath', 'Innerwear_imagePath 256x256',
       'Bottomwear_imagePath 256x256', 'Accessoire_imagePath 256x256',
       'Shoes_imagePath 256x256', 'Outerwear_imagePath 256x256',
       'valid_outfit'],
      dtype='object')

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6723 entries, 214761192 to 208373269
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Innerwear_imagePath           6723 non-null   object
 1   Bottomwear_imagePath          6723 non-null   object
 2   Accessoire_imagePath          5631 non-null   object
 3   Shoes_imagePath               3984 non-null   object
 4   Outerwear_imagePath           2179 non-null   object
 5   Innerwear_imagePath 256x256   6723 non-null   object
 6   Bottomwear_imagePath 256x256  6723 non-null   object
 7   Accessoire_imagePath 256x256  5631 non-null   object
 8   Shoes_imagePath 256x256       3984 non-null   object
 9   Outerwear_imagePath 256x256   2179 non-null   object
 10  valid_outfit                  6723 non-null   int64 
dtypes: int64(1), object(10)
memory usage: 630.3+ KB


In [17]:
for col in Columns_to_embed:
    df_not_none = df[df[col].notna()]
    df_not_none = df_not_none.head(10) #TODO: remove this line
    df.loc[df_not_none.index, f'{col}_Embedded'] = df_not_none[col].apply(lambda x: dinov2_classifier(f'../{x}'))

In [18]:
df.sample(10)

,Innerwear_imagePath,Bottomwear_imagePath,Accessoire_imagePath,Shoes_imagePath,Outerwear_imagePath,Innerwear_imagePath 256x256,Bottomwear_imagePath 256x256,Accessoire_imagePath 256x256,Shoes_imagePath 256x256,Outerwear_imagePath 256x256,valid_outfit,Innerwear_imagePath 256x256_Embedded,Bottomwear_imagePath 256x256_Embedded,Accessoire_imagePath 256x256_Embedded,Shoes_imagePath 256x256_Embedded,Outerwear_imagePath 256x256_Embedded
216019820,raw/images/216019820/1.jpg,raw/images/216019820/2.jpg,raw/images/216019820/4.jpg,raw/images/216019820/3.jpg,None,datasets/images_256x256/216019820/1.jpg,datasets/images_256x256/216019820/2.jpg,datasets/images_256x256/216019820/4.jpg,datasets/images_256x256/216019820/3.jpg,None,1,NaN,NaN,NaN,NaN,NaN
188190830,raw/images/188190830/1.jpg,raw/images/188190830/2.jpg,raw/images/188190830/4.jpg,None,None,datasets/images_256x256/188190830/1.jpg,datasets/images_256x256/188190830/2.jpg,datasets/images_256x256/188190830/4.jpg,None,None,0,NaN,NaN,NaN,NaN,NaN
215503296,raw/images/215503296/1.jpg,raw/images/215503296/2.jpg,raw/images/215503296/4.jpg,raw/images/215503296/3.jpg,None,datasets/images_256x256/215503296/1.jpg,datasets/images_256x256/215503296/2.jpg,datasets/images_256x256/215503296/4.jpg,datasets/images_256x256/215503296/3.jpg,None,1,NaN,NaN,NaN,NaN,NaN
191188526,raw/images/191188526/1.jpg,raw/images/191188526/3.jpg,raw/images/191188526/4.jpg,None,raw/images/191188526/2.jpg,datasets/images_256x256/191188526/1.jpg,datasets/images_256x256/191188526/3.jpg,datasets/images_256x256/191188526/4.jpg,None,datasets/images_256x256/191188526/2.jpg,1,NaN,NaN,NaN,NaN,NaN
210919469,raw/images/210919469/1.jpg,raw/images/210919469/3.jpg,None,None,raw/images/210919469/2.jpg,datasets/images_256x256/210919469/1.jpg,datasets/images_256x256/210919469/3.jpg,None,None,datasets/images_256x256/210919469/2.jpg,1,NaN,NaN,NaN,NaN,NaN
183020457,raw/images/183020457/1.jpg,raw/images/183020457/3.jpg,raw/images/183020457/5.jpg,None,raw/images/183020457/2.jpg,datasets/images_256x256/183020457/1.jpg,datasets/images_256x256/183020457/3.jpg,datasets/images_256x256/183020457/5.jpg,None,datasets/images_256x256/183020457/2.jpg,0,NaN,NaN,NaN,NaN,NaN
148802243,raw/images/148802243/1.jpg,raw/images/148802243/2.jpg,raw/images/148802243/5.jpg,None,None,datasets/images_256x256/148802243/1.jpg,datasets/images_256x256/148802243/2.jpg,datasets/images_256x256/148802243/5.jpg,None,None,0,NaN,NaN,NaN,NaN,NaN
173948288,raw/images/173948288/1.jpg,raw/images/173948288/2.jpg,raw/images/173948288/3.jpg,None,None,datasets/images_256x256/173948288/1.jpg,datasets/images_256x256/173948288/2.jpg,datasets/images_256x256/173948288/3.jpg,None,None,0,NaN,NaN,NaN,NaN,NaN
216783127,raw/images/216783127/1.jpg,raw/images/216783127/2.jpg,raw/images/216783127/4.jpg,None,None,datasets/images_256x256/216783127/1.jpg,datasets/images_256x256/216783127/2.jpg,datasets/images_256x256/216783127/4.jpg,None,None,1,NaN,NaN,NaN,NaN,NaN
125368926,raw/images/125368926/1.jpg,raw/images/125368926/2.jpg,raw/images/125368926/4.jpg,raw/images/125368926/3.jpg,None,datasets/images_256x256/125368926/1.jpg,datasets/images_256x256/125368926/2.jpg,datasets/images_256x256/125368926/4.jpg,datasets/images_256x256/125368926/3.jpg,None,0,NaN,NaN,NaN,NaN,NaN


In [19]:
df.columns

Index(['Innerwear_imagePath', 'Bottomwear_imagePath', 'Accessoire_imagePath',
       'Shoes_imagePath', 'Outerwear_imagePath', 'Innerwear_imagePath 256x256',
       'Bottomwear_imagePath 256x256', 'Accessoire_imagePath 256x256',
       'Shoes_imagePath 256x256', 'Outerwear_imagePath 256x256',
       'valid_outfit', 'Innerwear_imagePath 256x256_Embedded',
       'Bottomwear_imagePath 256x256_Embedded',
       'Accessoire_imagePath 256x256_Embedded',
       'Shoes_imagePath 256x256_Embedded',
       'Outerwear_imagePath 256x256_Embedded'],
      dtype='object')

In [20]:
dfEmbedded = df[['Innerwear_imagePath 256x256_Embedded', 'Bottomwear_imagePath 256x256_Embedded', 'Accessoire_imagePath 256x256_Embedded',
       'Shoes_imagePath 256x256_Embedded', 'Outerwear_imagePath 256x256_Embedded', "valid_outfit"]]

In [21]:
dfEmbedded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6723 entries, 214761192 to 208373269
Data columns (total 6 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   Innerwear_imagePath 256x256_Embedded   10 non-null     object
 1   Bottomwear_imagePath 256x256_Embedded  10 non-null     object
 2   Accessoire_imagePath 256x256_Embedded  10 non-null     object
 3   Shoes_imagePath 256x256_Embedded       10 non-null     object
 4   Outerwear_imagePath 256x256_Embedded   10 non-null     object
 5   valid_outfit                           6723 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 625.7+ KB


In [22]:
dfEmbedded["Innerwear_imagePath 256x256_Embedded"].iloc[0]

tensor([[-2.6804e-01,  1.8234e+00, -2.2911e-01,  2.5499e-01, -5.1269e+00,
         -2.6285e+00,  3.0815e-01, -3.1752e-01,  1.6644e+00,  1.6564e-01,
         -3.9223e-01,  2.4706e+00, -1.8322e+00,  2.3808e+00,  2.6750e+00,
          3.2331e+00,  9.8081e-01,  1.4576e+00,  5.4274e-01,  1.5695e+00,
         -5.9082e+00,  3.4384e+00,  2.7994e+00,  2.0053e+00, -1.5767e-01,
         -1.7529e+00, -4.9040e+00, -3.0555e-01, -8.4262e-02,  1.3014e+00,
         -3.6643e+00,  2.1681e-01, -1.9534e+00,  3.4836e+00, -8.2483e-01,
         -2.1847e+00,  2.6351e+00,  1.7316e+00,  3.3321e+00, -3.4122e+00,
         -6.1502e-01, -2.5803e-01, -2.8648e-01,  1.8518e+00, -5.2887e-02,
         -4.8509e-01, -2.1267e+00,  1.0014e+00,  1.9856e+00,  4.8047e+00,
          4.3354e+00,  2.2833e+00, -4.5096e+00, -8.1596e-02,  1.8166e+00,
          4.8219e+00,  3.3528e+00,  2.3925e+00,  2.6814e+00,  1.4379e+00,
          7.1504e-01,  1.9095e+00, -1.4072e+00, -2.5064e+00, -3.7175e+00,
         -4.4862e+00, -4.6084e-01, -1.

In [73]:
dfEmbedded.head()

,Innerwear_imagePath 256x256_Embedded,Bottomwear_imagePath 256x256_Embedded,Accessoire_imagePath 256x256_Embedded,Shoes_imagePath 256x256_Embedded,Outerwear_imagePath 256x256_Embedded,valid_outfit
214761192,"[[tensor(-0.2680), tensor(1.8234), tensor(-0.2...","[[tensor(-4.1725), tensor(3.4410), tensor(-0.7...","[[tensor(4.5869), tensor(0.8271), tensor(-1.44...","[[tensor(-1.1128), tensor(0.3236), tensor(-1.5...","[[tensor(1.5601), tensor(1.9414), tensor(0.289...",1
115479644,"[[tensor(3.1535), tensor(2.5599), tensor(1.474...","[[tensor(-1.8509), tensor(2.1373), tensor(2.63...","[[tensor(0.4958), tensor(-0.1515), tensor(1.71...",NaN,"[[tensor(-1.4221), tensor(2.7036), tensor(-1.1...",0
161222052,"[[tensor(1.1999), tensor(1.9961), tensor(2.264...","[[tensor(1.4631), tensor(-0.3849), tensor(0.61...","[[tensor(4.6151), tensor(1.2956), tensor(-6.35...",NaN,NaN,0
213330841,"[[tensor(3.1407), tensor(0.9145), tensor(1.569...","[[tensor(-4.0414), tensor(0.1743), tensor(3.40...","[[tensor(7.7586), tensor(1.5065), tensor(-3.12...","[[tensor(4.1571), tensor(1.6499), tensor(-2.21...",NaN,0
193320238,"[[tensor(2.1109), tensor(2.7435), tensor(-1.17...","[[tensor(-3.0450), tensor(0.7070), tensor(1.37...","[[tensor(1.6532), tensor(-0.6183), tensor(2.31...",NaN,"[[tensor(-1.3367), tensor(3.5270), tensor(2.12...",0


# Second Try with RENKU LAB

In [75]:
renkuDF = dfEmbedded.head()

In [76]:
renkuDF

,Innerwear_imagePath 256x256_Embedded,Bottomwear_imagePath 256x256_Embedded,Accessoire_imagePath 256x256_Embedded,Shoes_imagePath 256x256_Embedded,Outerwear_imagePath 256x256_Embedded,valid_outfit
214761192,"[[tensor(-0.2680), tensor(1.8234), tensor(-0.2...","[[tensor(-4.1725), tensor(3.4410), tensor(-0.7...","[[tensor(4.5869), tensor(0.8271), tensor(-1.44...","[[tensor(-1.1128), tensor(0.3236), tensor(-1.5...","[[tensor(1.5601), tensor(1.9414), tensor(0.289...",1
115479644,"[[tensor(3.1535), tensor(2.5599), tensor(1.474...","[[tensor(-1.8509), tensor(2.1373), tensor(2.63...","[[tensor(0.4958), tensor(-0.1515), tensor(1.71...",NaN,"[[tensor(-1.4221), tensor(2.7036), tensor(-1.1...",0
161222052,"[[tensor(1.1999), tensor(1.9961), tensor(2.264...","[[tensor(1.4631), tensor(-0.3849), tensor(0.61...","[[tensor(4.6151), tensor(1.2956), tensor(-6.35...",NaN,NaN,0
213330841,"[[tensor(3.1407), tensor(0.9145), tensor(1.569...","[[tensor(-4.0414), tensor(0.1743), tensor(3.40...","[[tensor(7.7586), tensor(1.5065), tensor(-3.12...","[[tensor(4.1571), tensor(1.6499), tensor(-2.21...",NaN,0
193320238,"[[tensor(2.1109), tensor(2.7435), tensor(-1.17...","[[tensor(-3.0450), tensor(0.7070), tensor(1.37...","[[tensor(1.6532), tensor(-0.6183), tensor(2.31...",NaN,"[[tensor(-1.3367), tensor(3.5270), tensor(2.12...",0


In [77]:
X_wine = renkuDF.drop(columns=["valid_outfit"]).values
y_wine = renkuDF["valid_outfit"].values

In [79]:
pca = PCA().fit(X_wine)
v_ratio = pca.explained_variance_ratio_

data = pd.DataFrame({'# of Features':range(1, len(v_ratio)+1), '% Variance explained':np.cumsum(v_ratio*100)})
data.plot(x=0, y=1, xticks=range(1, len(v_ratio)+1), grid=True, figsize=(10,8))

ValueError: only one element tensors can be converted to Python scalars

In [80]:
X_wine

array([[tensor([[-2.6804e-01,  1.8234e+00, -2.2911e-01,  2.5499e-01, -5.1269e+00,
                 -2.6285e+00,  3.0815e-01, -3.1752e-01,  1.6644e+00,  1.6564e-01,
                 -3.9223e-01,  2.4706e+00, -1.8322e+00,  2.3808e+00,  2.6750e+00,
                  3.2331e+00,  9.8081e-01,  1.4576e+00,  5.4274e-01,  1.5695e+00,
                 -5.9082e+00,  3.4384e+00,  2.7994e+00,  2.0053e+00, -1.5767e-01,
                 -1.7529e+00, -4.9040e+00, -3.0555e-01, -8.4262e-02,  1.3014e+00,
                 -3.6643e+00,  2.1681e-01, -1.9534e+00,  3.4836e+00, -8.2483e-01,
                 -2.1847e+00,  2.6351e+00,  1.7316e+00,  3.3321e+00, -3.4122e+00,
                 -6.1502e-01, -2.5803e-01, -2.8648e-01,  1.8518e+00, -5.2887e-02,
                 -4.8509e-01, -2.1267e+00,  1.0014e+00,  1.9856e+00,  4.8047e+00,
                  4.3354e+00,  2.2833e+00, -4.5096e+00, -8.1596e-02,  1.8166e+00,
                  4.8219e+00,  3.3528e+00,  2.3925e+00,  2.6814e+00,  1.4379e+00,
                

# FIRST Try with Online Help

In [24]:
#Combine everything into a single tensor

#Replace NaN with an Empty tensor
correct_size = torch.Size([1, 384])

zero_tensor = torch.zeros(correct_size)
empty_tensor = torch.empty(correct_size)
def replace_nan_with_tensor(x):
    return empty_tensor if pd.isna(x) else x

# Apply the function to the DataFrame
dfEmptyTensor = dfEmbedded.map(replace_nan_with_tensor)
dfZeroTensor = dfEmbedded.map(replace_nan_with_tensor)

In [25]:
dfEmptyTensor['Combined'] = dfEmptyTensor.apply(lambda row: torch.cat([row['Innerwear_imagePath 256x256_Embedded'], 
                                                         row['Bottomwear_imagePath 256x256_Embedded'], 
                                                         row['Accessoire_imagePath 256x256_Embedded'], 
                                                         row['Shoes_imagePath 256x256_Embedded'], 
                                                         row['Outerwear_imagePath 256x256_Embedded']]), axis=1)

In [26]:
dfZeroTensor['Combined'] = dfZeroTensor.apply(lambda row: torch.cat([row['Innerwear_imagePath 256x256_Embedded'], 
                                                         row['Bottomwear_imagePath 256x256_Embedded'], 
                                                         row['Accessoire_imagePath 256x256_Embedded'], 
                                                         row['Shoes_imagePath 256x256_Embedded'], 
                                                         row['Outerwear_imagePath 256x256_Embedded']]), axis=1)

In [27]:
# Either work with the Zero Tensor or Empty Tensor
df = dfZeroTensor[["Combined", "valid_outfit"]]

In [35]:
df = df.head(10)

In [43]:
df

,Combined,valid_outfit
214761192,"[[tensor(-0.2680), tensor(1.8234), tensor(-0.2...",1
115479644,"[[tensor(3.1535), tensor(2.5599), tensor(1.474...",0
161222052,"[[tensor(1.1999), tensor(1.9961), tensor(2.264...",0
213330841,"[[tensor(3.1407), tensor(0.9145), tensor(1.569...",0
193320238,"[[tensor(2.1109), tensor(2.7435), tensor(-1.17...",0


In [50]:
df['Combined']

214761192    [[tensor(-0.2680), tensor(1.8234), tensor(-0.2...
115479644    [[tensor(3.1535), tensor(2.5599), tensor(1.474...
161222052    [[tensor(1.1999), tensor(1.9961), tensor(2.264...
213330841    [[tensor(3.1407), tensor(0.9145), tensor(1.569...
193320238    [[tensor(2.1109), tensor(2.7435), tensor(-1.17...
Name: Combined, dtype: object

In [51]:
#Convert the tensor to a numpy array
df['Combined'] = df['Combined'].apply(lambda x: x.cpu().numpy())

In [56]:
print(df["Combined"].iloc[0])


[[-2.6803797e-01  1.8233917e+00 -2.2910976e-01 ... -6.7875081e-01
   5.4200560e-01 -2.5084805e-01]
 [-4.1724701e+00  3.4410243e+00 -7.8943664e-01 ...  4.0577907e-02
  -1.0160577e+00 -7.9886001e-01]
 [ 4.5868959e+00  8.2713240e-01 -1.4427762e+00 ...  1.2363329e+00
   2.9275560e+00 -1.6240718e+00]
 [-1.1128410e+00  3.2362920e-01 -1.5675240e+00 ...  2.2190554e+00
   2.7173033e+00  4.3451619e+00]
 [ 1.5601174e+00  1.9414332e+00  2.8913701e-01 ...  1.9452944e-03
   2.3705287e+00 -1.6797342e+00]]


In [63]:
df["Combined"].iloc[4].shape

(5, 384)

In [1]:
df

NameError: name 'df' is not defined